<a href="https://colab.research.google.com/github/pepetonof/UX-DL/blob/main/MadalineRuleII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

In [5]:
def bipolar(x):
    return np.where(x >= 0, 1, -1)

In [6]:
input_size = 2
hidden_size = 2
output_size = 1

X = np.array([[-1, -1], [-1, 1], [1, -1], [1, 1]]) # Valores de entrada
D = np.array([-1, 1, 1, -1])  # Salida esperada XOR

#pesos iniciales
#np.random.seed(0)
#W1 = np.array([[0.1, 0.2, -0.1],[0.3, -0.2, 0.2]])
#W2 = np.array([[0.1, 0.3, -0.4]])
#W1 = np.random.uniform(-1, 1, size = (hidden_size, input_size + 1))
#W2 = np.random.uniform(-1, 1, size = (output_size, hidden_size + 1))


In [20]:
class madaline:
    def __init__(self, input_size, hidden_size, output_size, lr=0.2):
        self.lr = lr
        # Inicializar pesos para la capa oculta (se incluye el peso de bias)
        #self.W1 = np.random.uniform(-1, 1, size = (hidden_size, input_size + 1))
        # Inicializar pesos para la capa de salida (incluyendo el bias de la capa oculta)
        #self.W2 = np.random.uniform(-1, 1, size = (output_size, hidden_size + 1))
        #self.W1 = np.array([[0.1, 0.2, -0.1],[0.3, -0.2, 0.2]])
        #self.W1 = np.array([[0.3, 0.0, 0.1],[0.3, -0.2, 0.2]])
        self.W1 = np.array([[0.1, 0.2, 0.1],[0.3, -0.2, 0.2]])
        #self.W2 = np.array([[0.3, 0.1, -0.6]])
        #self.W2 = np.array([[0.3, 0.1, -0.6]])
        self.W2 = np.array([[0.5, 0.3, -0.4]])
    def forward(self, x):
        if len(x.shape)==1:
            x = x.reshape(1,x.shape[0])
        rows = x.shape[0]
        ext = np.ones((rows, 1))
        if len(x.shape) == 1:
            x = x.reshape(x.shape[0],1)
        x_ext = np.hstack((ext,x))
        self.hidden_input = np.dot(x_ext, self.W1.T)  # vector de tamaño (hidden_size,)
        print('Entrada oculta', self.hidden_input)
        self.hidden_output = bipolar(self.hidden_input)
        print('Salida oculta', self.hidden_output)

        ext = np.ones((self.hidden_output.shape[0], 1))
        hidden_ext = np.hstack((ext,self.hidden_output))
        self.final_input = np.dot(hidden_ext, self.W2.T)
        print('Entrada final', self.final_input)
        self.final_output = bipolar(self.final_input)
        print('Salida final', self.final_output)
        return self.final_output

In [21]:
i=1
model = madaline(input_size, hidden_size, output_size)
output = model.forward(X[i])
error = D[i] - output
print('Error', error)

Entrada oculta [[0.  0.7]]
Salida oculta [[1 1]]
Entrada final [[0.4]]
Salida final [[1]]
Error [[0]]


In [ ]:
def train(model, X, D, epochs=10):
    for epoch in range(epochs):
        errores = 0
        for i in range(len(X)):
            x = X[i]
            print(x, x.shape)
            x = x.reshape(1, x.shape[0])
            print(x, x.shape)
            rows = x.shape[0]
            ext = np.ones((rows, 1))
            x_ext = np.hstack((x, ext))
            d = D[i]
            print(x_ext)
            y = model.forward(x_ext)
            error = d - y

            print(y, d)

            # Si la salida es incorrecta, aplicar la Regla MRII
            if error != 0:
                errores += 1
                # Seleccionar la neurona de la capa oculta con el nivel de confianza (valor absoluto del input)
                # más cercano a cero (la menos segura)
                min_index = np.argmin(np.abs(model.hidden_input))
                # Se agrega el bias a la entrada
                x_bias = np.insert(x, 0, 1)
                # Determinar el signo del ajuste: se busca invertir la salida de la neurona seleccionada
                delta = -np.sign(model.hidden_input[min_index]) if model.hidden_input[min_index] != 0 else 1
                # Actualizar los pesos de la neurona seleccionada en la capa oculta
                model.W1[min_index] += model.lr * delta * x_bias
                # Se vuelve a propagar la entrada para evaluar el cambio
                y_new = model.forward(x)
                # Si el error persiste, se actualizan los pesos de la capa de salida
                if y_new != d:
                    # Agregar bias a la salida oculta
                    hidden_bias = np.insert(model.hidden_output, 0, 1)
                    # Actualizar los pesos de la capa de salida usando una regla simple
                    model.W2 += model.lr * (d - y_new) * hidden_bias
        # Si en una época no se detectan errores, se ha alcanzado la convergencia
        if errores == 0:
            print(f"Convergencia alcanzada en la época {epoch+1}")
            break

In [ ]:
#Biblioteca numpy
import numpy as np

# Función de activación: hard-limiting
def hardlims(x):
    return np.where(x >= 0, 1, -1)

# Clase que implementa una red MADALINE con MRII para la compuerta XOR
class MadalineMRII:
    def __init__(self, input_size, hidden_size, output_size, lr=0.2):
        self.lr = lr
        # Inicializar pesos para la capa oculta (se incluye el peso de bias)
        self.W1 = np.random.uniform(-1, 1, (hidden_size, input_size + 1))
        # Inicializar pesos para la capa de salida (incluyendo el bias de la capa oculta)
        self.W2 = np.random.uniform(-1, 1, (output_size, hidden_size + 1))

    def forward(self, x):
        # x: vector de entrada de tamaño (input_size,)
        # Se añade el bias (valor 1) a la entrada
        x_bias = np.insert(x, 0, 1)
        # Cálculo de la entrada de la capa oculta
        self.hidden_input = np.dot(self.W1, x_bias)  # vector de tamaño (hidden_size,)
        # Salida de la capa oculta usando la función hardlims
        self.hidden_output = hardlims(self.hidden_input)
        # Añadir bias a la salida oculta
        hidden_bias = np.insert(self.hidden_output, 0, 1)
        # Cálculo de la entrada a la capa de salida
        self.final_input = np.dot(self.W2, hidden_bias)
        # Salida final de la red (también hardlims)
        self.final_output = hardlims(self.final_input)
        return self.final_output

    def train(self, X, D, epochs=10000):
        # X: matriz de entradas de tamaño (num_ejemplos, input_size)
        # D: vector columna de salidas deseadas de tamaño (num_ejemplos, 1)
        for epoch in range(epochs):
            errores = 0
            for i in range(len(X)):
                x = X[i]
                d = D[i]
                y = self.forward(x)
                error = d - y
                # Si la salida es incorrecta, aplicar la Regla MRII
                if error != 0:
                    errores += 1
                    # Seleccionar la neurona de la capa oculta con el nivel de confianza (valor absoluto del input)
                    # más cercano a cero (la menos segura)
                    min_index = np.argmin(np.abs(self.hidden_input))
                    # Se agrega el bias a la entrada
                    x_bias = np.insert(x, 0, 1)
                    # Determinar el signo del ajuste: se busca invertir la salida de la neurona seleccionada
                    delta = -np.sign(self.hidden_input[min_index]) if self.hidden_input[min_index] != 0 else 1
                    # Actualizar los pesos de la neurona seleccionada en la capa oculta
                    self.W1[min_index] += self.lr * delta * x_bias
                    # Se vuelve a propagar la entrada para evaluar el cambio
                    y_new = self.forward(x)
                    # Si el error persiste, se actualizan los pesos de la capa de salida
                    if y_new != d:
                        # Agregar bias a la salida oculta
                        hidden_bias = np.insert(self.hidden_output, 0, 1)
                        # Actualizar los pesos de la capa de salida usando una regla simple
                        self.W2 += self.lr * (d - y_new) * hidden_bias
            # Si en una época no se detectan errores, se ha alcanzado la convergencia
            if errores == 0:
                print(f"Convergencia alcanzada en la época {epoch+1}")
                break

# Definir los datos de la compuerta XOR
# Entradas: 2 neuronas
X = np.array([
    [-1, -1],
    [-1,  1],
    [ 1, -1],
    [ 1,  1]
])
# Salidas deseadas: 1 neurona (usando la convención bipolar)
D = np.array([
    [-1],
    [ 1],
    [ 1],
    [-1]
])

# Crear y entrenar el modelo
model = MadalineMRII(input_size=2, hidden_size=2, output_size=1, lr=0.2)
model.train(X, D, epochs=10000)

# Prueba de la red entrenada
print("Resultados para la compuerta XOR:")
for x in X:
    output = model.forward(x)
    print(f"Entrada: {x}, Salida: {output}")

Resultados para la compuerta XOR:
Entrada: [-1 -1], Salida: [1]
Entrada: [-1  1], Salida: [1]
Entrada: [ 1 -1], Salida: [-1]
Entrada: [1 1], Salida: [-1]


In [ ]:
model.W1

array([[-0.10237299,  0.63037873,  0.00552675],
       [-0.11023363, -0.3526904 ,  0.09178823]])

In [ ]:
model.W2

array([[-0.12482558, -0.016454  ,  0.12732552]])

In [ ]:
for x in X:
    print(x.shape, x)
    print(model.forward(x))

(2,) [-1 -1]
[1]
(2,) [-1  1]
[1]
(2,) [ 1 -1]
[-1]
(2,) [1 1]
[-1]


In [ ]:
model.forward([-1,-1])

array([1])

In [ ]:
len(X)

4